In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install mplcyberpunk
import numpy as np 
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.layers import Dense, Flatten, Dropout, Activation, Conv2D, MaxPooling2D, BatchNormalization
from zipfile import ZipFile
import pathlib, os, random, mplcyberpunk
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
with ZipFile('/kaggle/input/dogs-vs-cats-redux-kernels-edition/train.zip') as zf:
    zf.extractall('.')
    
with ZipFile('/kaggle/input/dogs-vs-cats-redux-kernels-edition/test.zip') as zf:
    zf.extractall('.')

In [ ]:
X, y = [], []

main_dir = "/kaggle/working/"
train_dir = "train"
path=os.path.join(main_dir,train_dir)
for i in os.listdir(path):
    classes = i.split('.')[0]
#     print(classes)
#     break
    img = cv2.imread(os.path.join(path, i), 0)
    img = cv2.resize(img, dsize=(100, 100))
    X.append(img)
    y.append(classes)

In [ ]:
dataInfo = pd.DataFrame(y)
dataInfo.columns = ['Target']
dictForClasses = {'cat': 0, 'dog': 1}
for i in range(25000):
    dataInfo.iloc[i, 0] = dictForClasses[dataInfo.iloc[i, 0]]

In [ ]:
dataInfo

In [ ]:
y = np.asarray(dataInfo.Target).astype('int64')
X = np.asarray(X).astype('float64')

In [ ]:
X = X.reshape(-1, 100, 100, 1)

In [ ]:
class BaseModel(tf.keras.Model):
    def __init__(self):
        super(BaseModel, self).__init__()
        self.C1 = Conv2D(32, (3 * 3), input_shape=(100, 100, 1), padding='same')
        self.B1 = BatchNormalization()
        self.A1 = Activation('relu')
        
        self.C2 = Conv2D(32, (3 * 3), padding='same')
        self.B2 = BatchNormalization()
        self.A2 = Activation('relu')
        self.P1 = MaxPooling2D(2, padding='same')
        self.Dr1 = Dropout(0.2)
        
#         self.C3 = Conv2D(16, (3 * 3))
#         self.B3 = BatchNormalization()
#         self.A3 = Activation('relu')
#         self.P2 = MaxPooling2D(2, padding='same')
#         self.Dr2 = Dropout(0.3)
        
        self.F1 = Flatten()
        self.B4 = BatchNormalization()
        self.D1 = Dense(64, activation='relu')
        self.Dr3 = Dropout(0.2)
        self.D2 = Dense(64, activation='relu')
        self.Dr4 = Dropout(0.3)
        self.D3 = Dense(2, activation='sigmoid')
        
    def call(self, x):
        x = self.C1(x)
        x = self.B1(x)
        x = self.A1(x)
        
        x = self.C2(x)
        x = self.B2(x)
        x = self.A2(x)
        x = self.P1(x)
        x = self.Dr1(x)
        
#         x = self.C3(x)
#         x = self.B3(x)
#         x = self.A3(x)
#         x = self.P2(x)
#         x = self.Dr2(x)

        x = self.F1(x)
        x = self.B4(x)
        x = self.D1(x)
#         x = self.Dr3(x)
        x = self.D2(x)
        x = self.Dr4(x)
        y = self.D3(x)
        return y
    
model = BaseModel()

model.compile(optimizer='adam',
             loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
             metrics=['accuracy'])

# checkpoint_save_path = './DogsAndCatsModel.ckpt'

# if os.path.exists(checkpoint_save_path + '.index'):
#     model.load_weights(checkpoint_save_path)
    
# cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_save_path,
#                                     save_best_only=True, save_weights_only=True)

# history = model.fit(X, y, epochs=20, batch_size=32,validation_split=0.2, 
#                     callbacks=[cp_callback])

history = model.fit(X, y, epochs=30, batch_size=32,validation_split=0.2 )

# file = open('./DogsAndCatsModel.txt', 'w')
# for v in model.trainable_variables:
#     file.write(str(v.name) + '\n')
#     file.write(str(v.shape) + '\n')
#     file.write(str(v.numpy()) + '\n')

# file.close()

model.summary()

In [ ]:
acc = history.history['accuracy']
loss = history.history['loss']

plt.figure(figsize=(12, 6))
plt.style.use('cyberpunk')
plt.subplot(1, 2, 1)
plt.plot(acc, label='Training Acc')
plt.ylim(0.5, 1)
plt.title('BaseModel Training Acc')
mplcyberpunk.add_glow_effects()
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(loss, label='Training Loss')
plt.title('BaseModel Training Loss')
mplcyberpunk.add_glow_effects()
plt.legend()

plt.show()